In [ ]:
from display import display_figures

from include.build_synth import buildData_fromRO as build
from include.build_synth import load_RO as load

from include.estim import Rt_MLE as estimMLE
from include.estim import Rt_PL as estimPL
from include.estim import Rt_Cori as estimCori
from include.estim import Rt_Joint as estimJ
from include.optim_tools import slidingMAD
from include.comparison_tools import SNR
from include.comparison_tools import jaccard_index

## Building synthetic data

Building according to the chosen configuration and initial number of cases.
See demo_buildSyntheticData.ipynb for more detailed building synthetic data explanations.

In [ ]:
configuration = 'I'  # Choice of configuration
firstCases = 100  # Choice of number of first cases on day one

# Loading realistic RTrue and OTrue
RTrue, OTrue = load.loadROconfig(configuration)
# Building synthetic data
datesBuilt, ZDataBuilt = build.buildData_anyRO(RTrue, OTrue, firstCases)
# Cropping RTrue for further comparisons
RTrueCrop = RTrue[1:]

In [ ]:
# Intermediate display of built data
fig, axes, formattedDatesBuilt = display_figures.display_dataBuilt(datesBuilt, ZDataBuilt, RTrue, OTrue)

## Comparison between R estimation methods

### Signal-to-Ratio Noise (dB)

$\mathsf{SNR}:=  10 \times \log_{10}\left( \frac{\lVert  \boldsymbol{\mathsf{R}}^{\text{true}}\rVert_2^2}{\lVert \widehat{\boldsymbol{\mathsf{R}}} - \boldsymbol{\mathsf{R}}^{\text{true}} \rVert_2^2}\right)$

where $\widehat{\boldsymbol{\mathsf{R}}}$ is the estimated $\boldsymbol{\mathsf{R}}$ with a chosen method.
### Jaccard index (%)

For $\boldsymbol{\mathsf{X}}, \boldsymbol{\mathsf{Y}} \in \lbrace 0, 1\rbrace^T, \, \,  \mathsf{Jac}^{\mathsf{g}}(\boldsymbol{\mathsf{X}},\boldsymbol{\mathsf{Y}}) := \frac{\sum_{t=1}^T \sqrt{\mathsf{X}^{\mathsf{g}}_t \times \mathsf{Y}^{\mathsf{g}}_t}}{\sum_{t=1}^T \mathsf{X}^{\mathsf{g}}_t + \mathsf{Y}^{\mathsf{g}}_t - \sqrt{\mathsf{X}^{\mathsf{g}}_t \times \mathsf{Y}^{\mathsf{g}}_t}}$

where $\mathsf{X}^{\mathsf{g}}$ (resp. $\mathsf{Y}^{\mathsf{g}}$) is the convolution between the binary vector $\boldsymbol{\mathsf{X}}$ (resp. $\boldsymbol{\mathsf{Y}}$) and a Gaussian distribution.
(see include/comparison_tools/jaccardIndex.py for more details)

### Maximum Likelihood Estimator
* $\mathsf{R}^{\mathsf{MLE}} \in \mathrm{arg\, min}_{\mathsf{R} \in  \mathbb{R} ^ {\mathsf{T}}} -\ln \mathsf{Likelihood}(\mathsf{Z} ^ {\mathsf{denoised}},  \mathsf{R})$

In [ ]:
# Analytic computation
R_MLE, datesMLE, dataMLE = estimMLE.Rt_MLE(datesBuilt, ZDataBuilt)

In [ ]:
# Computing SNR and Jaccard index
print('MLE Method indicators ---')
SNR_MLE = SNR.SignaltoNoiseRatio(RTrueCrop, R_MLE)
JaccardIndexMLE = jaccard_index.JaccardIndexREstim(RTrueCrop, R_MLE)
print("SNR = %.3f dB, associated Jaccard index = %.3f %%" % (SNR_MLE, JaccardIndexMLE))

In [ ]:
# Individual display
figMLE, axesMLE, formattedDatesMLE = display_figures.display_REstim(datesMLE, dataMLE, R_MLE, 'MLE', RTrue=RTrue)

### Cori Estimator
* $\mathsf{R}^{\mathsf{Cori}} = \frac{1 + \sum_{s = t -\tau}^{t}\mathsf{Z}_s}{5 + \sum_{s = t -\tau}^{t}(\Phi\star\mathsf{Z})_s}$

In [ ]:
# Analytic computation
R_Cori, datesCori, dataCori = estimCori.Rt_C(datesBuilt, ZDataBuilt)

In [ ]:
# Computing SNR and Jaccard index
print('Cori Method indicators ---')
SNR_Cori = SNR.SignaltoNoiseRatio(RTrueCrop, R_Cori)
JaccardIndexCori = jaccard_index.JaccardIndexREstim(RTrueCrop, R_Cori)
print("SNR = %.3f dB, associated Jaccard index = %.3f %%" % (SNR_Cori, JaccardIndexCori))

In [ ]:
# Individual display
figCori, axesCori, formattedDatesCori = display_figures.display_REstim(datesCori, dataCori, R_Cori, 'Cori', RTrue=RTrue)

### Penalized Likelihood estimation
1. $\mathsf{Z}^{\mathsf{denoised}} = \mathsf{slidingMAD}(\mathsf{Z}, \alpha)$

2. $\mathsf{R}^{\mathsf{PL}} \in \mathrm{arg\, min}_{\mathsf{R} \in  \mathbb{R} ^ {\mathsf{T}}} -\ln \mathsf{Likelihood}(\mathsf{Z}^{\mathsf{denoised}},  \mathsf{R}) + \mu_\mathsf{R} \| \boldsymbol{\mathsf{D}}_2\mathsf{R}\|_1$

In [ ]:
# Choice of denoising parameter alpha and regularization parameter muR
alpha = 0
muR = 50

# Denoising with sliding MAD then solving with Chambolle-Pock algorithm
denoisedData = slidingMAD.slidingMAD(ZDataBuilt, alpha)
R_PL, datesPL, denoisedDataPL = estimPL.Rt_PL(datesBuilt, denoisedData, muR)

In [ ]:
# Computing SNR and Jaccard index
print('PL Method indicators ---')
SNR_PL = SNR.SignaltoNoiseRatio(RTrueCrop, R_PL)
JaccardIndexPL = jaccard_index.JaccardIndexREstim(RTrueCrop, R_PL)
print("SNR = %.3f dB, associated Jaccard index = %.3f %%" % (SNR_PL, JaccardIndexPL))

In [ ]:
# Individual display
dataCropped = ZDataBuilt[1:]  # data[0] is used for burn-in
figPL, axesPL, formattedDatesPL = display_figures.display_REstim(datesPL, dataCropped, R_PL, 'PL', RTrue=RTrue)

### Joint estimation

* $\mathsf{R}^{\mathsf{Joint}}, \mathsf{O}^{\mathsf{Joint}} \in  \mathrm{arg\, min}_{\mathsf{R}, \mathsf{O} \in \mathbb{R}^{\mathsf{T}}} -\ln \mathsf{Likelihood}(\mathsf{Z}, \mathsf{R},  \mathsf{O}) + \lambda_\mathsf{R} \| \boldsymbol{\mathsf{D}}_2\mathsf{R}\|_1 + \lambda_\mathsf{O} \| \mathsf{O}\|_1$

In [ ]:
# Choice of denoising parameter alpha and regularization parameter muR
lambdaR = 3.5
lambdaO = 0.03

# Solving with Chambolle-Pock algorithm
R_Joint, O_Joint, datesJ, dataJ = estimJ.Rt_J(datesBuilt, ZDataBuilt, lambdaR, lambdaO)

In [ ]:
# Computing SNR and Jaccard index
print('J Method indicators ---')
SNR_Joint = SNR.SignaltoNoiseRatio(RTrueCrop, R_Joint)
JaccardIndexJoint = jaccard_index.JaccardIndexREstim(RTrueCrop, R_Joint)
print("SNR = %.3f dB, associated Jaccard index = %.3f %%" % (SNR_Joint, JaccardIndexJoint))

In [ ]:
# Individual display
figJoint, axesJoint, formattedDatesJoint = display_figures.display_REstim(datesJ, dataJ, R_Joint, 'Joint', RTrue=RTrue)

### Display comparison

In [ ]:
# Gather all methods
methods = ['MLE', 'Cori', 'PL', 'Joint']
REstimations = {'MLE': R_MLE,
                'Cori': R_Cori,
                'PL': R_PL,
                'Joint': R_Joint}

# Display comparison between methods
figComp, axesComp, formattedDatesComp = display_figures.display_REstim(datesJ, dataJ, REstimations, methods, RTrue=RTrue, comparison=True)